# Retrieval Augmented Question & Answering with Knowledge Bases for Amazon Bedrock & Amazon OpenSearch Serverless

> *This notebook should work well with the **`SageMaker Distribution 1.2`** image in SageMaker Studio* Jupyter Lab.

### Introduction
A Q&A chatbot powered by generative AI is designed to have natural conversations and answer questions on a wide range of topics.
It uses the LLM to understand questions and generate relevant and helpful responses. With generative AI capabilities, the chatbot can create unique responses instead of pulling from a database of pre-written responses. Overall, the goal is to have more human-like conversations that can educate, assist and to help improve user productivity.

While Q&A chatbots powered by the LLM are helpful in providing assistance across general topics, they struggle in providing information / assistance that involves domain specific knowledge, such as enterprise data that were not exposed to the model used in the training process. In order to make the Q&A chatbot understand enterprise data and to provide useful responses, 2 approaches are used in general to address the challenge:

1. Finetune the LLM model with enterprise data;
2. Integrate the LLM with enterprise knowledge through external databases (e.g. vector database). This approach is also referred as RAG (Retrieval Augmented Generation)

In this lab, we'll focus on building a Q&A chatbot using the RAG approach mentioned above. In particular, we'll explore a feature within Amazon Bedrock called [Knowledge Bases For Amazon Bedrock](https://aws.amazon.com/bedrock/knowledge-bases/) to help us quickly setup a vector database using [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/) and integrate with Amazon Bedrock foundation model without managing any infrasturcture.

### Use Case
A typical enterprise knowledge base involves large volume of data. In this lab, we'll use some sample gaming dataset provided by [IGDB](https://www.igdb.com/) as the source of the knowledge base. The data contains information about game titles and summaries. The Q&A chatbot will be used to integrate with the Knowledge Base to provide accurate answer based on user's question. 

These documents explain topics such as:
- Storyline of the game
- Game Identification
- Finding contextual Similar games

#### Persona
Let's assume a persona of a game user who's looking for information / guidance about the games available in the knowledge base repository.
The model will try to answer from the documents in natural language.


## Implementation
In order to follow the RAG approach this notebook integrates with Knowledge Bases for Amazon Bedrock. Specifically, we will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V2 available through Amazon Bedrock

- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

- **Vector Store**: Amazon OpenSearch Serverless available through Knowledge Bases for Amazon Bedrock 

- **Knowledge Base data** - Game Dataset in CSV stored in S3 bucket.

## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In [68]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.10.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached pyasn1-0.5.1-py2.py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 90.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 98.7 MB/s eta 0:00:00:00:0100:01
Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (705 kB)
Usi

In [69]:
import warnings
warnings.filterwarnings('ignore')

## Data Preparation
Let's first extract some of the files to build our knowledge base store. For this example we will be using the CSV file included in the data folder.

In [70]:
import sagemaker
sess = sagemaker.Session()
bucket = sess.default_bucket() # Set a default S3 bucket
prefix = 'bedrock/knowledgebase/'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


Now, let's extract the games csv data, and upload the csv file to S3 bucket so that we could ingest the data into Amazon OpenSearch Serverless via Knowledge Bases for Bedrock.

In [71]:
!cd data && tar -xvzf games.tar.gz && aws s3 cp games.csv s3://{bucket}/{prefix}

./
./games.csv
upload: ./games.csv to s3://sagemaker-us-east-1-349674815289/bedrock/knowledgebase/games.csv


**Note:** Please note the S3 URL, we'll use it in the next section.

# Create a Knowledge Base using Amazon Bedrock
The following section describes the steps to take in order to create a knowledge base in Bedrock.
For simplicity of the workshop, we are going to use the Amazon Bedrock console to configure all required components. 
You can also use the AWS SDK to achieve the same results. For information about using the AWS SDK for Agents, please refer to this [link](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent.html).

## Step by Step Instructions

1. Navigate to [Bedrock Console](https://console.aws.amazon.com/bedrock):

<img src="images/bedrock-console.png" alt="bedrock_console" style="width: 600px;"/>

2. Select Knowledge base from the left pane:

<img src="images/bedrock-console-kb.png" alt="bedrock_kb" style="width: 200px;"/>

3. Create a new Knowledge base:

<img src="images/bedrock-kb-create.png" alt="create_kb" style="width: 800px;"/>

4. Provide knowledge base details as followed:

**Name**: [your name]-genai-workshop-kb

**Description**: A sample knowledge base for gen AI workshop

**IAM permissions**: Create and use a new service role

<img src="images/bedrock-kb-detail.png" alt="bedrock_kb_detail" style="width: 700px;"/>

5. Add a new data source with the following details:

**Name**: [your name]-genai-workshop-kb-data-source

**S3 URI**: [ The S3 URI where the ``games.csv`` file was uploaded. (You can find the S3 URI in the previous cell) ]

**Advanced Settings**:

**KMS Key For transient data storage**: Use default KMS key

**Chunking Strategy**: Default Chunking

<img src="images/bedrock-kb-ds-detail.png" alt="bedrock_kb_ds_detail" style="width: 700px;"/>

6. Setup a vector store database

**Embeddings Model**: Titan Embeddings G1 - Text v1.2

**Vector Database**: Quick create a new vector store

<img src="images/bedrock-kb-vector-db-detail.png" alt="bedrock_kb_v_detail" style="width: 700px;"/>


7. Review and Create the Knowledge Base

<img src="images/bedrock-kb-create-final.png" alt="bedrock_kb_create_final" style="width: 700px;"/>

You'll see a status shows up at the top of the page, this should take a few seconds:

<img src="images/bedrock-kb-create-status.png" alt="bedrock_kb_create_final" style="width: 700px;"/>

When the vector DB is created successfully. You'll see the status bar turns Green. Click on the 'sync' button to sync the data source with the vector DB.

<img src="images/bedrock-kb-sync.png" alt="bedrock_kb_sync" style="width: 700px;"/>

The sync process could take a while depending on the volume of data. For our lab, it should take about 10 minutes. 

While waiting for the sync process, this might be a good time to take a break and resume when the sync is complete! 


## Knowledge Base Retrieval
We can use the Bedrock Agent SDK to perform similarity search to process a query and return the chunks of text without any LLM generating the response. 

First, let's retrieve the knowledge base ID so we could use it with the SDK. You can the Knowledge Base ID on the overview page of the Knowledge Base when you created it. Here's a screenshot that shows where the ID is located:

<img src="images/bedrock-kb-overview.png" alt="bedrock_kb_overview" style="width: 450px;"/>

Define a runtime bedrock agent client

In [72]:
agent_runtime_client = boto3.client('bedrock-agent-runtime')
knowledgebase_id = "TWV0RKFM9I" # use the knowledge base ID from the console shown in the previous step.

In [73]:
response = agent_runtime_client.retrieve(
    knowledgeBaseId=knowledgebase_id,
    retrievalQuery={
        'text': 'Dance dance revolution'
    },
    retrievalConfiguration={
        'vectorSearchConfiguration': {
            'numberOfResults': 3  # Shows the top 3 results
        }
    }
)

prints out the top 3 matching documents

In [74]:
for text, score in [ (x['content']['text'], x['score']) for x in response['retrievalResults'] ]:
    print(f"==> Document Text: {text}, Score: {score}")

==> Document Text: An updated version, Dance Dance Revolution 2ndMix Link Version, was released to Japanese arcades on April 28, 1999. This version came with a PlayStation memory card reader, installed in the middle of the arcade cabinet. It supports cards that have Link Data from the home version of Dance Dance Revolution, allowing each player to save high scores and play custom step edits. 2ndMix Link Version adds five new songs to the game, two from the home version and three new licenses, for a total of 37 songs. Future Dance Dance Revolution releases in Japan, up to and including Dance Dance Revolution Extreme, integrated Link Data functionality in-game. However, these required different home games to produce different Link Data formats: 3rdMix, 4thMix, 4thMix Plus, 5thMix and New Version." 122565,Humanity,"A unique blend of puzzle-solving and action-platforming. Play a lone Shiba Inu charged with commanding massive, marching crowds to the goal in Story Mode, or browse a wide arra

## Generative Question Answering
In generative question-answering (GQA), we pass our question to the Claude-2 but instruct it to base the answer on the information returned from our knowledge base.
Typically, in order to integrate knowledge base with an LLM for a chatbot application, you would need to setup, build and manage a QA retriever that connects both components. With Knowledge Bases for Amazon Bedrock, you simply use Bedrock API to send the question, Bedrock is responsible for handling the connectivity between LLM and the Knowledge base components, orchestrate the interactions and returns the results. It helps improves developer productivity as there is no infrastructure to manage. 

First let's list all the model IDs available to find the Claude-2 model ARN. We'll need it for invoking the agent and knowledge base.  

In [75]:
bedrock_agent = boto3.client("bedrock")

In [76]:
for model_id, model_name, model_arn in [ (x['modelId'], x['modelName'], x['modelArn']) for x in bedrock_agent.list_foundation_models()['modelSummaries']]:
  print(f"model ID: {model_id}, model name: {model_name}, model_arn: {model_arn}")

model ID: amazon.titan-tg1-large, model name: Titan Text Large, model_arn: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large
model ID: amazon.titan-image-generator-v1:0, model name: Titan Image Generator G1, model_arn: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0
model ID: amazon.titan-image-generator-v1, model name: Titan Image Generator G1, model_arn: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1
model ID: amazon.titan-embed-g1-text-02, model name: Titan Text Embeddings v2, model_arn: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-g1-text-02
model ID: amazon.titan-text-lite-v1:0:4k, model name: Titan Text G1 - Lite, model_arn: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-lite-v1:0:4k
model ID: amazon.titan-text-lite-v1, model name: Titan Text G1 - Lite, model_arn: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-lite-v1
model ID: amazon.titan-text-express-v1:0

In [78]:
response = client.retrieve_and_generate(
    input={
        'text': 'show me similar games like "Dance dance revolution" '
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': knowledgebase_id,
            'modelArn': claude_v2_model_id
        }
    }
)

Here's the response

In [81]:
print(response['output']['text'])

Dance Dance Revolution is a pioneering rhythm and dance game where players hit colored arrows with their feet to musical and visual cues. Similar games inspired by DDR include: Pump It Up by Andamiro In the Groove by Roxor


## Conclusion
Congratulations on completing this moduel on retrieval augmented generation! This is an important technique that combines the power of large language models with the precision of retrieval methods. By augmenting generation with relevant retrieved examples, the responses we recieved become more coherent, consistent and grounded. You should feel proud of learning this innovative approach. I'm sure the knowledge you've gained will be very useful for building creative and engaging language generation systems. Well done!

In the above implementation of RAG based Question Answering we have explored the following concepts and how to implement them using Amazon Bedrock and it's LangChain integration.

- Creating a knowledge base using Knowledge Bases for Bedrock
- Loads documents and generating embeddings to create a vector store (Amazon Opensearch Serveless) managed by Amazon Bedrock Knowledge Base. 
- Retrieving similar documents to the question
- Use Bedrock agent SDK to retrieval and generate a human friendly response based on user question.

# Thank You